In [1]:
import pandas as pd

df = pd.read_csv('data/games.csv')

df.head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [2]:
import chess

def pgn_to_move_index(pgn_moves):

    board = chess.Board()


    moves_san = pgn_moves.split()


    for move_san in moves_san[:-1]:
        try:
            move = board.parse_san(move_san)
            board.push(move)
        except:

            continue


    last_move_san = moves_san[-1]
    try:
        last_move = board.parse_san(last_move_san)
        from_square = last_move.from_square
        to_square = last_move.to_square


        return from_square * 64 + to_square
    except:

        return None
  

In [3]:
df['moves_input'] = df['moves'].apply(lambda moves: moves.rsplit(' ', 1)[0])

In [4]:
df['pred_output'] = df['moves'].apply(pgn_to_move_index)

In [5]:
df['moves'][0]

'd4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4'

In [6]:
df['moves_input'][0]

'd4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5'

In [7]:
import chess
import chess.pgn
import io


def moves_to_board_state(moves):
    pgn = io.StringIO(moves)
    game = chess.pgn.read_game(pgn)
    board = game.board()


    for move in game.mainline_moves():
        board.push(move)

    return board


df['final_board_state'] = df['moves_input'].apply(moves_to_board_state)


print(df['final_board_state'].iloc[0])

r n b q k . n r
p p . . . . p p
. . p . p . . .
b . . . . . . .
. . . P . . . .
. . N . . N . .
P P . . P P P P
R . B Q K B . R


In [8]:
import numpy as np
import torch


def board_to_tensor(board):
    tensor = np.zeros((12, 8, 8))
    piece_map = board.piece_map()

    piece_to_index = {
        chess.PAWN: 0,
        chess.KNIGHT: 1,
        chess.BISHOP: 2,
        chess.ROOK: 3,
        chess.QUEEN: 4,
        chess.KING: 5
    }

    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_idx = piece_to_index[piece.piece_type]
        if piece.color == chess.WHITE:
            tensor[piece_idx, row, col] = 1
        else:
            tensor[piece_idx + 6, row, col] = 1

    return torch.tensor(tensor, dtype=torch.float)


df['input_tensor'] = df['final_board_state'].apply(board_to_tensor)


print(df['input_tensor'].iloc[0])

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
       

In [9]:
from torch.utils.data import DataLoader, Dataset

class ChessDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_tensor = self.data.iloc[idx]['input_tensor']
        move_label = self.data.iloc[idx]['pred_output']
        return input_tensor, move_label


train_loader = DataLoader(ChessDataset(df), batch_size=32, shuffle=True)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


class ChessMovePredictionModel(nn.Module):
    def __init__(self):
        super(ChessMovePredictionModel, self).__init__()
        self.conv1 = nn.Conv2d(12, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 4096)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(-1, 128 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = ChessMovePredictionModel()


optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()


train_loader = DataLoader(ChessDataset(df), batch_size=32, shuffle=True)


def train(model, train_loader, optimizer, loss_fn, epochs=30):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")


train(model, train_loader, optimizer, loss_fn, epochs=30)

Epoch 1, Loss: 7.343059082730916
Epoch 2, Loss: 6.523734306414542
Epoch 3, Loss: 5.328731215171266
Epoch 4, Loss: 4.152989243776604
Epoch 5, Loss: 3.040569417404406
Epoch 6, Loss: 2.0222063781351944
Epoch 7, Loss: 1.1733443598237714
Epoch 8, Loss: 0.592322650744299
Epoch 9, Loss: 0.2805558861941812
Epoch 10, Loss: 0.16709105419128706
Epoch 11, Loss: 0.12879311296482834
Epoch 12, Loss: 0.14453768384816687
Epoch 13, Loss: 0.17270481008353036
Epoch 14, Loss: 0.1374162506234893
Epoch 15, Loss: 0.09806650493775258
Epoch 16, Loss: 0.08901453465282727
Epoch 17, Loss: 0.09491278314584216
Epoch 18, Loss: 0.10454568349306236
Epoch 19, Loss: 0.09360952021416159
Epoch 20, Loss: 0.07887814328500754
Epoch 21, Loss: 0.07380081241350864
Epoch 22, Loss: 0.0822841012946024
Epoch 23, Loss: 0.08643506914520178
Epoch 24, Loss: 0.07299947295832912
Epoch 25, Loss: 0.07842043096937114
Epoch 26, Loss: 0.06525995149358846
Epoch 27, Loss: 0.06532047493491906
Epoch 28, Loss: 0.0758278197959573
Epoch 29, Loss: 0.0

In [11]:
def evaluate(model, val_loader, loss_fn):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()


            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    avg_loss = total_loss / len(val_loader)
    accuracy = correct_predictions / total_predictions
    print(f"Val Loss: {avg_loss}, Accuracy: {accuracy * 100:.2f}%")

In [12]:
from sklearn.model_selection import train_test_split


train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [13]:

val_loader = DataLoader(ChessDataset(val_df), batch_size=32, shuffle=False)

evaluate(model, val_loader, loss_fn)

Val Loss: 0.03555329321153251, Accuracy: 99.03%


In [14]:

torch.save(model.state_dict(), 'chess_model.pth')


model = ChessMovePredictionModel()
model.load_state_dict(torch.load('chess_model.pth'))


/tmp/ipykernel_14659/1046560708.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('chess_model.pth'))


<All keys matched successfully>